<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/QAS_BERT_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 
import os 
import sys 
import json
from platform import python_version

import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding,Dense,MultiHeadAttention

import tensorflow_hub as hub
!pip install -q tensorflow-text

import tensorflow_text as text

     |████████████████████████████████| 3.4MB 5.3MB/s 


In [2]:
print('Python Version : ' ,python_version())
print('tensorflow version : ', tf.__version__)
print('pandas version : ', pd.__version__)
print('numpy version : ', np.__version__)
print('tf_hub version : ', hub.__version__)


Python Version :  3.7.10
tensorflow version :  2.4.1
pandas version :  1.1.5
numpy version :  1.19.5
tf_hub version :  0.11.0


In [3]:
train_path = keras.utils.get_file("train.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json")
eval_path = keras.utils.get_file("eval.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json")
with open(train_path) as f: raw_train_data = json.load(f)
with open(eval_path) as f: raw_eval_data = json.load(f)
# max_seq_length = 384

4857856/4854279 [==============================] - 0s 0us/step


In [4]:
print(raw_train_data.keys())
print(raw_train_data['data'][0].keys())
print(raw_train_data['data'][0]['title'][0:])
print(raw_train_data['data'][1]['title'][0:])

dict_keys(['data', 'version'])
dict_keys(['title', 'paragraphs'])
University_of_Notre_Dame
Beyoncé


In [5]:
print(raw_train_data['data'][3]['paragraphs'][0].keys())

dict_keys(['context', 'qas'])


In [6]:
print(raw_train_data['data'][0]['paragraphs'][0]['qas'])

[{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}], 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'id': '5733be284776f41900661182'}, {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ'}], 'question': 'What is in front of the Notre Dame Main Building?', 'id': '5733be284776f4190066117f'}, {'answers': [{'answer_start': 279, 'text': 'the Main Building'}], 'question': 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?', 'id': '5733be284776f41900661180'}, {'answers': [{'answer_start': 381, 'text': 'a Marian place of prayer and reflection'}], 'question': 'What is the Grotto at Notre Dame?', 'id': '5733be284776f41900661181'}, {'answers': [{'answer_start': 92, 'text': 'a golden statue of the Virgin Mary'}], 'question': 'What sits on top of the Main Building at Notre Dame?', 'id': '5733be284776f4190066117e'}]


In [7]:
print(raw_train_data['data'][0]['title'])
print(raw_train_data.keys())
print(raw_train_data['data'][0].keys())
print(raw_train_data['data'][0]['paragraphs'][0]['qas'][1]['answers'][0]['text'])
x = (raw_train_data['data'][0]['paragraphs'][0]['context'])
x

University_of_Notre_Dame
dict_keys(['data', 'version'])
dict_keys(['title', 'paragraphs'])
a copper statue of Christ


'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [8]:
raw_train_data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start']

515

In [9]:
bert_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess_layer = hub.load(bert_preprocess)

bert_model = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
bert_layer = hub.KerasLayer(bert_model,trainable=True)

In [15]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
print(f'BERT vocab is stored at     : {vocab_file}')
print(f'BERT model is case sensitive: {do_lower_case}')

BERT vocab is stored at     : b'/tmp/tfhub_modules/89e0d78326196f7e9dd113bd0eecf4c9fb022f37/assets/vocab.txt'
BERT model is case sensitive: True


In [22]:
def load_vocab(vocab_file):
  """Load a vocabulary file into a list."""
  vocab = []
  with tf.io.gfile.GFile(vocab_file, "r") as reader:
    while True:
      token = reader.readline()
      if not token: break
      token = token.strip()
      vocab.append(token)
  return vocab
def create_vocab_table(vocab, num_oov=1):
  """Create a lookup table for a vocabulary"""
  vocab_values = tf.range(tf.size(vocab, out_type=tf.int64), dtype=tf.int64)
  init = tf.lookup.KeyValueTensorInitializer(keys=vocab, values=vocab_values, key_dtype=tf.string, value_dtype=tf.int64)
  vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov, lookup_key_dtype=tf.string)
  return vocab_table

vocab_lookup_table = create_vocab_table(vocab)
vocab = load_vocab(vocab_file)

In [28]:
vocab[9196]

'divorced'

In [ ]:
def text_maker (data):
  squad_examples = []
  full_context = []
  full_question = []
  full_answer = []
  title = []
  starting_index = []
  context_question =[]
  new_line =[]
  ending_index = []

  for item in data["data"]:
   
      title.append(item['title'])
      for para in item["paragraphs"]:
          context = para["context"]
          # full_context.append(context)

          

          for qa in para["qas"]:
              question = qa["question"]
              x = np.hstack((context,question))
              new_line.append(x)

              

              if "answers" in qa:
                  answer_text = qa["answers"][0]["text"]
                  full_answer.append(answer_text)
                  all_answers = [_["text"] for _ in qa["answers"]]
                  start_char_idx = qa["answers"][0]["answer_start"]
                  end_char_idx = start_char_idx + len(answer_text)
                  # print(start_char_idx)
                  starting_index.append(start_char_idx)
                  ending_index.append(end_char_idx)
                # squad_eg = Sample(question, context, start_char_idx, answer_text, all_answers)
            # else:
                # squad_eg = Sample(question, context)
            # squad_eg.preprocess()
            # squad_examples.append(squad_eg)
  x  = pd.DataFrame(new_line)
  # x.rename(columns ={0:"context", 1:"questions"})
  x['start_index'] = starting_index
  x['end_index'] = ending_index
  
  return (x)

In [ ]:
batch_size = 1

In [ ]:
train_data= (text_maker(raw_train_data)).rename(columns ={0:"context", 1:"questions"})
test_data = (text_maker(raw_eval_data)).rename(columns ={0:"context", 1:"questions"})
# all_data[0].head()

In [ ]:
train_data.head(3)

,context,questions,start_index,end_index
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,541
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,213
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,296


In [ ]:
test_data.head(3)

,context,questions,start_index,end_index
0,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,177,191
1,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,249,266
2,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,403,426


In [ ]:
train_dataset_context=tf.data.Dataset.from_tensor_slices(train_data['context'])
train_dataset_context =train_dataset_context.batch(batch_size)
train_dataset_ques = tf.data.Dataset.from_tensor_slices(train_data['questions'])
train_dataset_ques = train_dataset_ques.batch(batch_size)

In [ ]:
def bert_tokenizer(x):
  return (bert_preprocess_layer.tokenize(x))

def token_combiner(x,y):
  return (bert_preprocess_layer.bert_pack_inputs([x,y]))

def bert_encoder(x):
  return (bert_layer(x))

In [ ]:
training_context_tokens = []
for i in train_dataset_context:
  x =bert_tokenizer(i)
  training_context_tokens.append(x)

training_ques_tokens = []
for i in train_dataset_ques:
  x =bert_tokenizer(i)
  training_ques_tokens.append(x)


In [ ]:
combined_train_tokens = []
for i in range(len(training_ques_tokens)):
  x = token_combiner(training_context_tokens[i],training_ques_tokens[i])
  # y = bert_encoder(x)
  combined_train_tokens.append(x)

In [ ]:
# bert_encoder(combined_train_tokens)

In [ ]:
total_input_mask = []
total_input_word_ids = []
total_input_type_ids = []
for i in range(len(combined_train_tokens)):
  total_input_mask.append((combined_train_tokens[i]['input_mask']).numpy())
  total_input_word_ids.append(combined_train_tokens[i]['input_word_ids'])
  total_input_type_ids.append(combined_train_tokens[i]['input_type_ids'])

In [ ]:
total_input_mask_conc= tf.reshape(total_input_mask , [len(total_input_mask),128])
total_input_word_ids_conc = tf.reshape(total_input_word_ids ,[len(total_input_word_ids),128])
total_input_type_ids_conc = tf.reshape(total_input_type_ids ,[len(total_input_type_ids),128])

In [ ]:
dataset_dict = {
    'input_mask': total_input_mask_conc,
    'input_type_ids': total_input_word_ids_conc , 
    'input_word_ids': total_input_type_ids_conc
}

In [ ]:
# max_seq_length = 384
# input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_word_ids')
# input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_mask')
# input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_type_ids')
# bert_model = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
# bert_layer = hub.KerasLayer(bert_model,trainable=True)
# pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])

In [ ]:
# model = keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=[start_probs, end_probs])
# loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# optimizer = keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
# model.compile(optimizer=optimizer, loss=[loss, loss])
# model.fit(dataset_dict, train_data['start_index'], epochs=2, batch_size=8)

In [ ]:
dataset_dict2 = {
    'input_mask': total_input_mask_conc[0],
    'input_type_ids': total_input_word_ids_conc[0] , 
    'input_word_ids': total_input_type_ids_conc[0]
}
dataset_dict2

{'input_mask': <tf.Tensor: shape=(128,), dtype=int32, numpy=
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
 array([  101,  6549,  2135,  1010,  1996,  2082,  2038,  1037,  3234,
         2839,  1012, 10234,  1996,  2364,  2311,  1005,  1055,  2751,
         8514,  2003,  1037,  3585,  6231,  1997,  1996,  6261,  2984,
         1012,  3202,  1999,  2392,  1997,  1996,  2364,  2311,  1998,
         5307,  2009,  1010,  2003,  1037,  6967,  6231,  1997,  4828,
         2007,  2608,  2039, 14995,  6924,  2007,  1996,  5722,  1000,


In [ ]:
bert_layer([
            tf.reshape(dataset_dict2['input_mask'],(-1,1) ),
            tf.reshape(dataset_dict2['input_type_ids'],(-1,1) ),
            tf.reshape(dataset_dict2['input_word_ids'],(-1,1) )
            ])

ValueError: ignored

In [ ]:
(dataset_dict2['input_mask'])

<tf.Tensor: shape=(128,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)>

In [ ]:
(tf.reshape(dataset_dict2['input_mask'],(-1,1)))

TensorShape([128, 1])